# Chapter 7: Functions and Tasks

Functions and tasks are essential constructs in SystemVerilog that allow you to create reusable blocks of code, improving modularity and maintainability. While they serve similar purposes, they have distinct characteristics and use cases.

## Function Declarations and Calls

Functions in SystemVerilog are subroutines that return a value and execute in zero simulation time. They are ideal for computational operations and combinational logic.

## Basic Function Syntax

```systemverilog
function [return_type] function_name ([arguments]);
    // Function body
    return return_value;
endfunction
```

### Simple Function Examples

```systemverilog
module function_examples;
    
    // Function to add two integers
    function int add(int a, int b);
        return a + b;
    endfunction
    
    // Function to find maximum of two values
    function int max(int x, int y);
        if (x > y)
            return x;
        else
            return y;
    endfunction
    
    // Function with bit vector operations
    function logic [7:0] reverse_bits(logic [7:0] data);
        logic [7:0] result;
        for (int i = 0; i < 8; i++) begin
            result[i] = data[7-i];
        end
        return result;
    endfunction
    
    initial begin
        int result1, result2, result3;
        logic [7:0] original = 8'b10110100;
        logic [7:0] reversed;
        
        result1 = add(15, 25);           // Returns 40
        result2 = max(100, 50);          // Returns 100
        reversed = reverse_bits(original); // Returns 8'b00101101
        
        $display("Add result: %d", result1);
        $display("Max result: %d", result2);
        $display("Original: %b, Reversed: %b", original, reversed);
    end
    
endmodule
```

### Functions with Different Return Types

```systemverilog
module function_types;
    
    // Function returning a structure
    typedef struct {
        int quotient;
        int remainder;
    } div_result_t;
    
    function div_result_t divide(int dividend, int divisor);
        div_result_t result;
        result.quotient = dividend / divisor;
        result.remainder = dividend % divisor;
        return result;
    endfunction
    
    // Function returning an array
    function logic [3:0] [7:0] create_pattern(logic [7:0] base);
        logic [3:0] [7:0] pattern;
        for (int i = 0; i < 4; i++) begin
            pattern[i] = base << i;
        end
        return pattern;
    endfunction
    
    initial begin
        div_result_t div_res;
        logic [3:0] [7:0] pattern;
        
        div_res = divide(17, 5);
        pattern = create_pattern(8'b00001111);
        
        $display("17/5 = %d remainder %d", div_res.quotient, div_res.remainder);
        
        for (int i = 0; i < 4; i++) begin
            $display("Pattern[%d]: %b", i, pattern[i]);
        end
    end
    
endmodule
```

## Task Declarations and Calls

Tasks are subroutines that can consume simulation time and don't return values directly. They can have input, output, and inout arguments, making them suitable for complex operations and time-consuming activities.

### Basic Task Syntax

```systemverilog
task task_name ([arguments]);
    // Task body
endtask
```

### Task Examples

```systemverilog
module task_examples;
    
    logic clk = 0;
    logic [7:0] data;
    logic valid;
    
    // Generate clock
    always #5 clk = ~clk;
    
    // Task to wait for a number of clock cycles
    task wait_cycles(int num_cycles);
        repeat(num_cycles) @(posedge clk);
    endtask
    
    // Task to send data with handshaking
    task send_data(input logic [7:0] send_data, output logic done);
        data = send_data;
        valid = 1'b1;
        @(posedge clk);
        valid = 1'b0;
        done = 1'b1;
        @(posedge clk);
        done = 1'b0;
    endtask
    
    // Task with multiple outputs
    task analyze_data(input logic [7:0] input_data, 
                      output int ones_count, 
                      output int zeros_count,
                      output logic parity);
        ones_count = 0;
        zeros_count = 0;
        
        for (int i = 0; i < 8; i++) begin
            if (input_data[i])
                ones_count++;
            else
                zeros_count++;
        end
        
        parity = ^input_data; // XOR reduction for parity
    endtask
    
    initial begin
        logic done;
        int ones, zeros;
        logic par;
        
        // Wait for some cycles
        wait_cycles(3);
        
        // Send some data
        send_data(8'hA5, done);
        $display("Data sent, done = %b", done);
        
        // Analyze data
        analyze_data(8'b11010110, ones, zeros, par);
        $display("Ones: %d, Zeros: %d, Parity: %b", ones, zeros, par);
        
        #100 $finish;
    end
    
endmodule
```

## Automatic vs. Static Lifetime

The lifetime of variables in functions and tasks can be either static (default) or automatic. This affects how variables are allocated and whether they retain values between calls.

### Static Lifetime (Default)

```systemverilog
module static_lifetime;
    
    // Static function - variables retain values between calls
    function int counter();
        int count = 0;  // Initialized only once
        count++;
        return count;
    endfunction
    
    // Static task
    task static_task();
        static int call_count = 0;
        call_count++;
        $display("Static task called %d times", call_count);
    endtask
    
    initial begin
        $display("Counter: %d", counter()); // Prints 1
        $display("Counter: %d", counter()); // Prints 2
        $display("Counter: %d", counter()); // Prints 3
        
        static_task(); // Prints: Static task called 1 times
        static_task(); // Prints: Static task called 2 times
    end
    
endmodule
```

### Automatic Lifetime

```systemverilog
module automatic_lifetime;
    
    // Automatic function - fresh variables for each call
    function automatic int factorial(int n);
        if (n <= 1)
            return 1;
        else
            return n * factorial(n-1); // Recursive call possible
    endfunction
    
    // Automatic task
    task automatic print_sequence(int start, int count);
        for (int i = 0; i < count; i++) begin
            $display("Value: %d", start + i);
            #10; // Can consume time
        end
    endtask
    
    initial begin
        $display("5! = %d", factorial(5)); // Prints 120
        
        // Multiple concurrent task calls
        fork
            print_sequence(10, 3);
            print_sequence(20, 3);
        join
    end
    
endmodule
```

## Pass by Reference

SystemVerilog supports passing arguments by reference using the `ref` keyword, allowing functions and tasks to modify the original variables.

### Pass by Reference Examples

```systemverilog
module pass_by_reference;
    
    // Function with reference arguments
    function automatic void swap(ref int a, ref int b);
        int temp = a;
        a = b;
        b = temp;
    endfunction
    
    // Task to initialize an array by reference
    task automatic init_array(ref int arr[10], input int init_value);
        for (int i = 0; i < 10; i++) begin
            arr[i] = init_value + i;
        end
    endtask
    
    // Function to modify a structure by reference
    typedef struct {
        int x, y;
        string name;
    } point_t;
    
    function automatic void move_point(ref point_t p, int dx, int dy);
        p.x += dx;
        p.y += dy;
    endfunction
    
    initial begin
        int x = 10, y = 20;
        int my_array[10];
        point_t my_point = '{100, 200, "PointA"};
        
        $display("Before swap: x=%d, y=%d", x, y);
        swap(x, y);
        $display("After swap: x=%d, y=%d", x, y);
        
        init_array(my_array, 50);
        $display("Array elements: %p", my_array);
        
        $display("Before move: %s at (%d,%d)", my_point.name, my_point.x, my_point.y);
        move_point(my_point, 15, -25);
        $display("After move: %s at (%d,%d)", my_point.name, my_point.x, my_point.y);
    end
    
endmodule
```

## Return Statements in Functions

Functions must return a value, and the return statement determines what value is returned. You can have multiple return statements in a function.

### Multiple Return Statements

```systemverilog
module return_statements;
    
    // Function with multiple return points
    function automatic string grade_letter(int score);
        if (score >= 90)
            return "A";
        else if (score >= 80)
            return "B"; 
        else if (score >= 70)
            return "C";
        else if (score >= 60)
            return "D";
        else
            return "F";
    endfunction
    
    // Function with early return for error checking
    function automatic real safe_divide(real dividend, real divisor);
        if (divisor == 0.0) begin
            $error("Division by zero attempted");
            return 0.0; // Early return for error case
        end
        return dividend / divisor;
    endfunction
    
    // Function with complex logic and multiple returns
    function automatic int find_first_one(logic [31:0] data);
        for (int i = 0; i < 32; i++) begin
            if (data[i] == 1'b1)
                return i; // Return position of first '1'
        end
        return -1; // Return -1 if no '1' found
    endfunction
    
    initial begin
        string letter;
        real result;
        int position;
        
        letter = grade_letter(85);
        $display("Score 85 gets grade: %s", letter);
        
        result = safe_divide(10.0, 3.0);
        $display("10.0 / 3.0 = %f", result);
        
        result = safe_divide(5.0, 0.0); // Will show error
        
        position = find_first_one(32'h00008000);
        $display("First '1' found at position: %d", position);
        
        position = find_first_one(32'h00000000);
        $display("First '1' found at position: %d", position);
    end
    
endmodule
```

## Void Functions

Void functions don't return a value and are similar to tasks, but they execute in zero simulation time and cannot contain timing control statements.

### Void Function Examples

```systemverilog
module void_functions;
    
    int global_counter = 0;
    logic [7:0] memory [256];
    
    // Void function to increment global counter
    function automatic void increment_counter(int step);
        global_counter += step;
    endfunction
    
    // Void function to initialize memory
    function automatic void init_memory(logic [7:0] pattern);
        for (int i = 0; i < 256; i++) begin
            memory[i] = pattern ^ i[7:0];
        end
    endfunction
    
    // Void function for debug printing
    function automatic void debug_print(string msg, int value);
        $display("[DEBUG %0t] %s: %d", $time, msg, value);
    endfunction
    
    // Void function with reference parameter
    function automatic void reset_array(ref int arr[]);
        foreach(arr[i]) begin
            arr[i] = 0;
        end
    endfunction
    
    initial begin
        int test_array[5] = '{1, 2, 3, 4, 5};
        
        debug_print("Initial counter", global_counter);
        
        increment_counter(5);
        debug_print("After increment", global_counter);
        
        init_memory(8'hAA);
        debug_print("Memory[0]", memory[0]);
        debug_print("Memory[1]", memory[1]);
        debug_print("Memory[255]", memory[255]);
        
        $display("Before reset: %p", test_array);
        reset_array(test_array);
        $display("After reset: %p", test_array);
    end
    
endmodule
```

## Best Practices and Guidelines

### When to Use Functions vs. Tasks

**Use Functions when:**
- You need to return a single value
- The operation is purely combinational
- No timing control is needed
- The operation should complete in zero simulation time

**Use Tasks when:**
- You need multiple outputs
- Timing control is required
- The operation may consume simulation time
- You need to model sequential behavior

### Function and Task Design Guidelines

```systemverilog
module design_guidelines;
    
    // Good: Pure function with clear purpose
    function automatic int absolute_value(int value);
        return (value < 0) ? -value : value;
    endfunction
    
    // Good: Task with clear interface and timing
    task automatic wait_for_ready(ref logic ready_signal, input int timeout_cycles);
        int cycle_count = 0;
        while (!ready_signal && cycle_count < timeout_cycles) begin
            @(posedge clk);
            cycle_count++;
        end
        if (cycle_count >= timeout_cycles) begin
            $error("Timeout waiting for ready signal");
        end
    endtask
    
    // Good: Function with appropriate use of reference
    function automatic void normalize_vector(ref real vector[3]);
        real magnitude = $sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2);
        if (magnitude != 0.0) begin
            vector[0] /= magnitude;
            vector[1] /= magnitude;
            vector[2] /= magnitude;
        end
    endfunction
    
endmodule
```

## Summary

Functions and tasks are powerful constructs in SystemVerilog that enable code reuse and modular design:

- **Functions** return values, execute in zero time, and are ideal for combinational logic
- **Tasks** can have multiple outputs, consume time, and are suited for sequential operations
- **Automatic lifetime** creates fresh variables for each call and enables recursion
- **Static lifetime** (default) preserves variable values between calls
- **Pass by reference** allows modification of original variables
- **Void functions** provide task-like behavior without return values but in zero time

Understanding when and how to use functions and tasks effectively will greatly improve your SystemVerilog code organization and reusability.

# SystemVerilog Functions and Tasks - Example Index

## Function Declarations and Calls

#### Mathematical Functions
Basic arithmetic operations like factorial, power, greatest common divisor

In [2]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_1__mathematical_functions/"
files = ["mathematical_calculator.sv", "mathematical_calculator_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// mathematical_calculator.sv
module mathematical_calculator ();               // Mathematical functions design

  // Function to calculate factorial
  function automatic integer calculate_factorial;
    input integer number_input;
    integer factorial_result;
    begin
      if (number_input <= 1)
        factorial_result = 1;
      else
        factorial_result = number_input * calculate_factorial(number_input - 1);
      calculate_factorial = factorial_result;
    end
  endfunction

  // Function to calculate power (base^exponent)
  function automatic integer calculate_power;
    input integer base_value;
    input integer exponent_value;
    integer power_result;
    integer loop_counter;
    begin
      power_result = 1;
      for (loop_counter = 0; loop_counter < exponent_value; loop_counter = loop_counter + 1)
        power_result = power_result * base_value;
      calculate_power = power_result;
    end
  endfunction

  // Function to calculate greatest common divisor using Euclidean algorithm
  function automatic integer calculate_gcd;
    input integer first_number;
    input integer second_number;
    integer temp_remainder;
    begin
      while (second_number != 0) begin
        temp_remainder = first_number % second_number;
        first_number = second_number;
        second_number = temp_remainder;
      end
      calculate_gcd = first_number;
    end
  endfunction

  initial begin
    $display();
    $display("=== Mathematical Functions Demonstration ===");
    
    // Test factorial function
    $display("Factorial Tests:");
    $display("  5! = %0d", calculate_factorial(5));
    $display("  0! = %0d", calculate_factorial(0));
    $display("  3! = %0d", calculate_factorial(3));
    
    // Test power function
    $display("Power Tests:");
    $display("  2^3 = %0d", calculate_power(2, 3));
    $display("  5^2 = %0d", calculate_power(5, 2));
    $display("  10^0 = %0d", calculate_power(10, 0));
    
    // Test GCD function
    $display("Greatest Common Divisor Tests:");
    $display("  GCD(48, 18) = %0d", calculate_gcd(48, 18));
    $display("  GCD(100, 25) = %0d", calculate_gcd(100, 25));
    $display("  GCD(17, 13) = %0d", calculate_gcd(17, 13));
    
    $display();
  end

endmodule
```

```systemverilog
// mathematical_calculator_testbench.sv
module math_calculator_testbench;  // Testbench for mathematical calculator

  mathematical_calculator MATH_CALCULATOR_INSTANCE();  // Instantiate mathematical calculator

  // Test variables
  integer test_factorial_result;
  integer test_power_result;
  integer test_gcd_result;

  initial begin
    // Dump waves for simulation
    $dumpfile("math_calculator_testbench.vcd");
    $dumpvars(0, math_calculator_testbench);
    
    #1;  // Wait for design to initialize
    
    $display();
    $display("=== Testbench Verification ===");
    
    // Verify factorial calculations
    test_factorial_result = MATH_CALCULATOR_INSTANCE.calculate_factorial(4);
    $display("Testbench: 4! = %0d (Expected: 24)", test_factorial_result);
    
    // Verify power calculations  
    test_power_result = MATH_CALCULATOR_INSTANCE.calculate_power(3, 4);
    $display("Testbench: 3^4 = %0d (Expected: 81)", test_power_result);
    
    // Verify GCD calculations
    test_gcd_result = MATH_CALCULATOR_INSTANCE.calculate_gcd(60, 48);
    $display("Testbench: GCD(60, 48) = %0d (Expected: 12)", test_gcd_result);
    
    $display();
    $display("Mathematical functions testbench completed successfully!");
    $display();
    
    #10;  // Wait before finishing
    $finish;
  end

endmodule
```

Verilator Simulation Output:

=== Mathematical Functions Demonstration ===
Factorial Tests:
  5! = 120
  0! = 1
  3! = 6
Power Tests:
  2^3 = 8
  5^2 = 25
  10^0 = 1
Greatest Common Divisor Tests:
  GCD(48, 18) = 6
  GCD(100, 25) = 25
  GCD(17, 13) = 1


=== Testbench Verification ===
Testbench: 4! = 24 (Expected: 24)
Testbench: 3^4 = 81 (Expected: 81)
Testbench: GCD(60, 48) = 12 (Expected: 12)

Mathematical functions testbench completed successfully!

Process finished with return code: 0
Removing Chapter_7_examples/example_1__mathematical_functions/obj_dir directory...
Chapter_7_examples/example_1__mathematical_functions/obj_dir removed successfully.


0

#### String Manipulation Functions
Functions for string processing, parsing, and formatting operations

In [7]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_2__string_manipulation/"
files = ["string_processor.sv", "string_processor_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// string_processor.sv
module string_processor ();
  
  // String manipulation functions demonstration
  function automatic string format_name(string first_name, string last_name);
    return $sformatf("%s %s", first_name, last_name);
  endfunction
  
  function automatic string extract_extension(string filename);
    int dot_position;
    dot_position = filename.len() - 1;
    
    // Find the last dot in filename
    while (dot_position >= 0 && filename[dot_position] != ".") begin
      dot_position--;
    end
    
    if (dot_position >= 0)
      return filename.substr(dot_position + 1, filename.len() - 1);
    else
      return "no_extension";
  endfunction
  
  function automatic string reverse_string(string input_text);
    string reversed_text = "";
    for (int i = input_text.len() - 1; i >= 0; i--) begin
      reversed_text = {reversed_text, input_text[i]};
    end
    return reversed_text;
  endfunction
  
  function automatic int count_vowels(string text);
    int vowel_count = 0;
    string lowercase_text;
    
    lowercase_text = text.tolower();
    
    for (int i = 0; i < lowercase_text.len(); i++) begin
      case (lowercase_text[i])
        "a", "e", "i", "o", "u": vowel_count++;
        default: ; // Non-vowel characters, do nothing
      endcase
    end
    return vowel_count;
  endfunction
  
  initial begin
    string first_name = "Alice";
    string last_name = "Johnson";
    string full_name;
    string filename = "report.txt";
    string file_extension;
    string sample_text = "SystemVerilog";
    string reversed_text;
    int vowel_count;
    
    $display("=== String Manipulation Functions Demo ===");
    $display();
    
    // Format name demonstration
    full_name = format_name(first_name, last_name);
    $display("Name formatting:");
    $display("  First: %s, Last: %s", first_name, last_name);
    $display("  Full name: %s", full_name);
    $display();
    
    // File extension extraction
    file_extension = extract_extension(filename);
    $display("File extension extraction:");
    $display("  Filename: %s", filename);
    $display("  Extension: %s", file_extension);
    $display();
    
    // String reversal
    reversed_text = reverse_string(sample_text);
    $display("String reversal:");
    $display("  Original: %s", sample_text);
    $display("  Reversed: %s", reversed_text);
    $display();
    
    // Vowel counting
    vowel_count = count_vowels(sample_text);
    $display("Vowel counting:");
    $display("  Text: %s", sample_text);
    $display("  Vowel count: %0d", vowel_count);
    $display();
  end

endmodule
```

```systemverilog
// string_processor_testbench.sv
module string_processor_testbench;
  
  // Instantiate the string processor design
  string_processor STRING_PROCESSOR_INSTANCE();
  
  // Additional testbench verification
  task automatic verify_string_functions();
    string test_first = "Bob";
    string test_last = "Smith";
    string expected_full = "Bob Smith";
    string actual_full;
    
    string test_filename = "data.csv";
    string expected_ext = "csv";
    string actual_ext;
    
    string test_word = "hello";
    string expected_reverse = "olleh";
    string actual_reverse;
    
    int expected_vowels = 2; // "e" and "o" in "hello"
    int actual_vowels;
    
    $display("=== Testbench Verification ===");
    $display();
    
    // Test name formatting
    actual_full = STRING_PROCESSOR_INSTANCE.format_name(test_first, test_last);
    $display("Name Format Test:");
    $display("  Expected: %s", expected_full);
    $display("  Actual:   %s", actual_full);
    $display("  Result:   %s", (actual_full == expected_full) ? "PASS" : "FAIL");
    $display();
    
    // Test file extension extraction
    actual_ext = STRING_PROCESSOR_INSTANCE.extract_extension(test_filename);
    $display("Extension Test:");
    $display("  Expected: %s", expected_ext);
    $display("  Actual:   %s", actual_ext);
    $display("  Result:   %s", (actual_ext == expected_ext) ? "PASS" : "FAIL");
    $display();
    
    // Test string reversal
    actual_reverse = STRING_PROCESSOR_INSTANCE.reverse_string(test_word);
    $display("Reverse Test:");
    $display("  Expected: %s", expected_reverse);
    $display("  Actual:   %s", actual_reverse);
    $display("  Result:   %s", (actual_reverse == expected_reverse) ? "PASS" : "FAIL");
    $display();
    
    // Test vowel counting
    actual_vowels = STRING_PROCESSOR_INSTANCE.count_vowels(test_word);
    $display("Vowel Count Test:");
    $display("  Expected: %0d", expected_vowels);
    $display("  Actual:   %0d", actual_vowels);
    $display("  Result:   %s", (actual_vowels == expected_vowels) ? "PASS" : "FAIL");
    $display();
    
  endtask
  
  initial begin
    // Dump waves for analysis
    $dumpfile("string_processor_testbench.vcd");
    $dumpvars(0, string_processor_testbench);
    
    #1; // Wait for design to initialize
    
    $display("Hello from string processor testbench!");
    $display();
    
    // Run verification tests
    verify_string_functions();
    
    $display("=== Test Summary ===");
    $display("String manipulation functions tested successfully!");
    $display();
    
    #5; // Additional time before finish
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== String Manipulation Functions Demo ===

Name formatting:
  First: Alice, Last: Johnson
  Full name: Alice Johnson

File extension extraction:
  Filename: report.txt
  Extension: txt

String reversal:
  Original: SystemVerilog
  Reversed: golireVmetsyS

Vowel counting:
  Text: SystemVerilog
  Vowel count: 4

Hello from string processor testbench!

=== Testbench Verification ===

Name Format Test:
  Expected: Bob Smith
  Actual:   Bob Smith
  Result:   PASS

Extension Test:
  Expected: csv
  Actual:   csv
  Result:   PASS

Reverse Test:
  Expected: olleh
  Actual:   olleh
  Result:   PASS

Vowel Count Test:
  Expected: 2
  Actual:   2
  Result:   PASS

=== Test Summary ===
String manipulation functions tested successfully!

Process finished with return code: 0
Removing Chapter_7_examples/example_2__string_manipulation/obj_dir directory...
Chapter_7_examples/example_2__string_manipulation/obj_dir removed successfully.


0

#### Bit Manipulation Functions
Functions for bit operations, masking, shifting, and bit counting

In [3]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_3__bit_manipulation_functions/"
files = [
    "bit_manipulation_functions.sv",
    "bit_manipulation_functions_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// bit_manipulation_functions.sv
module bit_manipulation_functions ();

  // Function to count number of set bits (population count)
  function automatic int count_ones(input logic [31:0] data_word);
    int bit_count = 0;
    for (int bit_index = 0; bit_index < 32; bit_index++) begin
      if (data_word[bit_index]) bit_count++;
    end
    return bit_count;
  endfunction

  // Function to reverse bits in a byte
  function automatic logic [7:0] reverse_byte_bits(input logic [7:0] byte_in);
    logic [7:0] reversed_byte;
    for (int bit_pos = 0; bit_pos < 8; bit_pos++) begin
      reversed_byte[7-bit_pos] = byte_in[bit_pos];
    end
    return reversed_byte;
  endfunction

  // Function to create bit mask with specified width and position
  function automatic logic [31:0] create_bit_mask(input int mask_width, 
                                                   input int start_position);
    logic [31:0] generated_mask = 0;
    for (int mask_bit = 0; mask_bit < mask_width; mask_bit++) begin
      generated_mask[start_position + mask_bit] = 1'b1;
    end
    return generated_mask;
  endfunction

  // Function to extract bit field from word
  function automatic logic [31:0] extract_bit_field(input logic [31:0] source_word,
                                                     input int field_width,
                                                     input int field_position);
    logic [31:0] extraction_mask;
    extraction_mask = create_bit_mask(field_width, field_position);
    return (source_word & extraction_mask) >> field_position;
  endfunction

  // Function to perform circular left shift
  function automatic logic [15:0] circular_left_shift(input logic [15:0] shift_data,
                                                       input int shift_amount);
    int effective_shift = shift_amount % 16;
    return (shift_data << effective_shift) | (shift_data >> (16 - effective_shift));
  endfunction

  initial begin
    logic [31:0] test_word = 32'hA5C3_F0E1;
    logic [7:0] test_byte = 8'b1010_0011;
    logic [15:0] rotation_data = 16'hBEEF;
    
    $display("=== Bit Manipulation Functions Demo ===");
    $display();
    
    // Test count_ones function
    $display("Count ones in 0x%08h: %0d bits", test_word, count_ones(test_word));
    
    // Test reverse_byte_bits function  
    $display("Reverse bits in 0x%02h: 0x%02h", test_byte, 
             reverse_byte_bits(test_byte));
    
    // Test create_bit_mask function
    $display("Create 4-bit mask at position 8: 0x%08h", 
             create_bit_mask(4, 8));
    
    // Test extract_bit_field function
    $display("Extract 8 bits from position 16 of 0x%08h: 0x%02h", 
             test_word, extract_bit_field(test_word, 8, 16));
    
    // Test circular_left_shift function
    $display("Circular left shift 0x%04h by 4 positions: 0x%04h", 
             rotation_data, circular_left_shift(rotation_data, 4));
    
    $display();
  end

endmodule
```

```systemverilog
// bit_manipulation_functions_testbench.sv
module bit_manipulation_functions_testbench;

  // Instantiate design under test
  bit_manipulation_functions BIT_MANIP_DUT();

  // Test variables
  logic [31:0] test_data_word;
  logic [7:0] test_input_byte;
  logic [15:0] shift_test_data;
  int expected_ones_count;
  logic [7:0] expected_reversed_byte;
  logic [31:0] expected_mask_value;
  logic [31:0] expected_extracted_field;
  logic [15:0] expected_shifted_result;

  initial begin
    // Setup waveform dump
    $dumpfile("bit_manipulation_functions_testbench.vcd");
    $dumpvars(0, bit_manipulation_functions_testbench);
    
    $display("=== Comprehensive Bit Manipulation Testing ===");
    $display();
    
    // Test Case 1: Count ones function verification
    test_data_word = 32'hFFFF_0000;  // 16 ones, 16 zeros
    expected_ones_count = 16;
    #1;
    if (BIT_MANIP_DUT.count_ones(test_data_word) == expected_ones_count) begin
      $display("PASS: count_ones(0x%08h) = %0d", 
               test_data_word, BIT_MANIP_DUT.count_ones(test_data_word));
    end else begin
      $display("FAIL: count_ones(0x%08h) = %0d, expected %0d", 
               test_data_word, BIT_MANIP_DUT.count_ones(test_data_word), 
               expected_ones_count);
    end
    
    // Test Case 2: Byte bit reversal verification
    test_input_byte = 8'b1100_0011;  // 0xC3
    expected_reversed_byte = 8'b1100_0011;  // Same when palindromic
    #1;
    if (BIT_MANIP_DUT.reverse_byte_bits(test_input_byte) == expected_reversed_byte) begin
      $display("PASS: reverse_byte_bits(0x%02h) = 0x%02h", 
               test_input_byte, BIT_MANIP_DUT.reverse_byte_bits(test_input_byte));
    end else begin
      $display("FAIL: reverse_byte_bits(0x%02h) = 0x%02h, expected 0x%02h", 
               test_input_byte, BIT_MANIP_DUT.reverse_byte_bits(test_input_byte),
               expected_reversed_byte);
    end
    
    // Test Case 3: Bit mask creation verification
    expected_mask_value = 32'h0000_0F00;  // 4 bits at position 8
    #1;
    if (BIT_MANIP_DUT.create_bit_mask(4, 8) == expected_mask_value) begin
      $display("PASS: create_bit_mask(4, 8) = 0x%08h", 
               BIT_MANIP_DUT.create_bit_mask(4, 8));
    end else begin
      $display("FAIL: create_bit_mask(4, 8) = 0x%08h, expected 0x%08h", 
               BIT_MANIP_DUT.create_bit_mask(4, 8), expected_mask_value);
    end
    
    // Test Case 4: Bit field extraction verification
    test_data_word = 32'hABCD_EF12;
    expected_extracted_field = 32'h0000_00CD;  // Bits [15:8] = 0xCD
    #1;
    if (BIT_MANIP_DUT.extract_bit_field(test_data_word, 8, 8) == 
        expected_extracted_field) begin
      $display("PASS: extract_bit_field(0x%08h, 8, 8) = 0x%08h", 
               test_data_word, BIT_MANIP_DUT.extract_bit_field(test_data_word, 8, 8));
    end else begin
      $display("FAIL: extract_bit_field(0x%08h, 8, 8) = 0x%08h, expected 0x%08h", 
               test_data_word, BIT_MANIP_DUT.extract_bit_field(test_data_word, 8, 8),
               expected_extracted_field);
    end
    
    // Test Case 5: Circular shift verification
    shift_test_data = 16'hF000;  // 1111_0000_0000_0000
    expected_shifted_result = 16'h000F;  // After 4-bit left shift
    #1;
    if (BIT_MANIP_DUT.circular_left_shift(shift_test_data, 4) == 
        expected_shifted_result) begin
      $display("PASS: circular_left_shift(0x%04h, 4) = 0x%04h", 
               shift_test_data, 
               BIT_MANIP_DUT.circular_left_shift(shift_test_data, 4));
    end else begin
      $display("FAIL: circular_left_shift(0x%04h, 4) = 0x%04h, expected 0x%04h", 
               shift_test_data, 
               BIT_MANIP_DUT.circular_left_shift(shift_test_data, 4),
               expected_shifted_result);
    end
    
    $display();
    $display("=== Bit Manipulation Testing Complete ===");
    
    #1;
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Bit Manipulation Functions Demo ===

Count ones in 0xa5c3f0e1: 16 bits
Reverse bits in 0xa3: 0xc5
Create 4-bit mask at position 8: 0x00000f00
Extract 8 bits from position 16 of 0xa5c3f0e1: 0xc3
Circular left shift 0xbeef by 4 positions: 0xeefb

=== Comprehensive Bit Manipulation Testing ===

PASS: count_ones(0xffff0000) = 16
PASS: reverse_byte_bits(0xc3) = 0xc3
PASS: create_bit_mask(4, 8) = 0x00000f00
FAIL: extract_bit_field(0xabcdef12, 8, 8) = 0x000000ef, expected 0x000000cd
PASS: circular_left_shift(0xf000, 4) = 0x000f

=== Bit Manipulation Testing Complete ===
Process finished with return code: 0
Removing Chapter_7_examples/example_3__bit_manipulation_functions/obj_dir directory...
Chapter_7_examples/example_3__bit_manipulation_functions/obj_dir removed successfully.


0

#### CRC Calculator Function
Function to calculate cyclic redundancy check for data integrity

In [6]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_7_examples/example_4__crc_calculator/"
files = [
    "crc_calculator_design.sv",
    "crc_calculator_design_testbench.sv",
]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// crc_calculator_design.sv
module crc_calculator_design_module (
  input  logic        clock_signal,
  input  logic        reset_active_low,
  input  logic        data_valid_input,
  input  logic [7:0]  data_byte_input,
  output logic [7:0]  crc_checksum_output,
  output logic        crc_calculation_complete
);

  // CRC-8 polynomial: x^8 + x^2 + x^1 + 1 (0x07)
  parameter logic [7:0] CRC_POLYNOMIAL_CONSTANT = 8'h07;
  
  // Internal registers for CRC calculation
  logic [7:0] crc_register_current;
  logic [7:0] crc_register_next;
  logic       calculation_in_progress;

  // CRC calculation logic
  always_comb begin
    crc_register_next = crc_register_current;
    
    if (data_valid_input) begin
      // XOR input data with current CRC
      crc_register_next = crc_register_current ^ data_byte_input;
      
      // Process each bit through the polynomial
      for (int bit_position = 0; bit_position < 8; bit_position++) begin
        if (crc_register_next[7]) begin
          crc_register_next = (crc_register_next << 1) ^ CRC_POLYNOMIAL_CONSTANT;
        end else begin
          crc_register_next = crc_register_next << 1;
        end
      end
    end
  end

  // Sequential logic for state updates
  always_ff @(posedge clock_signal or negedge reset_active_low) begin
    if (!reset_active_low) begin
      crc_register_current     <= 8'h00;
      calculation_in_progress  <= 1'b0;
      crc_calculation_complete <= 1'b0;
    end else begin
      crc_register_current <= crc_register_next;
      
      if (data_valid_input) begin
        calculation_in_progress  <= 1'b1;
        crc_calculation_complete <= 1'b0;
      end else if (calculation_in_progress) begin
        calculation_in_progress  <= 1'b0;
        crc_calculation_complete <= 1'b1;
      end else begin
        crc_calculation_complete <= 1'b0;
      end
    end
  end

  // Output assignment
  assign crc_checksum_output = crc_register_current;

  // Display messages for debugging
  initial $display("CRC Calculator Design Module Initialized");
  
  always @(posedge crc_calculation_complete) begin
    $display("CRC calculation complete! Checksum: 0x%02h", 
             crc_checksum_output);
  end

endmodule
```

```systemverilog
// crc_calculator_design_testbench.sv
module crc_calculator_testbench_module;

  // Testbench signals with descriptive names
  logic        clock_signal_tb;
  logic        reset_active_low_tb;
  logic        data_valid_input_tb;
  logic [7:0]  data_byte_input_tb;
  logic [7:0]  crc_checksum_output_tb;
  logic        crc_calculation_complete_tb;

  // Test data array for CRC calculation
  logic [7:0] test_data_array [4] = '{8'hAB, 8'hCD, 8'hEF, 8'h12};
  integer     data_index_counter;

  // Instantiate the CRC calculator design under test
  crc_calculator_design_module CRC_CALCULATOR_INSTANCE (
    .clock_signal              (clock_signal_tb),
    .reset_active_low          (reset_active_low_tb),
    .data_valid_input          (data_valid_input_tb),
    .data_byte_input           (data_byte_input_tb),
    .crc_checksum_output       (crc_checksum_output_tb),
    .crc_calculation_complete  (crc_calculation_complete_tb)
  );

  // Clock generation - 10ns period
  always #5 clock_signal_tb = ~clock_signal_tb;

  // Main test sequence
  initial begin
    // Initialize waveform dump
    $dumpfile("crc_calculator_testbench_module.vcd");
    $dumpvars(0, crc_calculator_testbench_module);
    
    // Display test start message
    $display();
    $display("=== CRC Calculator Testbench Starting ===");
    $display();

    // Initialize all signals
    clock_signal_tb      = 1'b0;
    reset_active_low_tb  = 1'b1;
    data_valid_input_tb  = 1'b0;
    data_byte_input_tb   = 8'h00;
    data_index_counter   = 0;

    // Apply reset sequence
    $display("Applying reset sequence...");
    #10 reset_active_low_tb = 1'b0;  // Assert reset
    #20 reset_active_low_tb = 1'b1;  // Release reset
    #10;

    // Test CRC calculation with sample data
    $display("Starting CRC calculation with test data:");
    for (data_index_counter = 0; data_index_counter < 4; 
         data_index_counter++) begin
      
      $display("Processing byte %0d: 0x%02h", 
               data_index_counter, test_data_array[data_index_counter]);
      
      // Present data to CRC calculator
      data_byte_input_tb  = test_data_array[data_index_counter];
      data_valid_input_tb = 1'b1;
      #10;
      
      // Remove data valid signal
      data_valid_input_tb = 1'b0;
      #10;
      
      // Wait for calculation completion
      wait (crc_calculation_complete_tb);
      #5;
    end

    // Final CRC result display
    $display();
    $display("Final CRC checksum result: 0x%02h", crc_checksum_output_tb);
    $display();

    // Test reset functionality
    $display("Testing reset functionality...");
    #10 reset_active_low_tb = 1'b0;
    #10 reset_active_low_tb = 1'b1;
    #10;
    
    $display("CRC after reset: 0x%02h", crc_checksum_output_tb);
    $display();

    // Complete testbench execution
    $display("=== CRC Calculator Testbench Complete ===");
    $display();
    #50;
    $finish;
  end

  // Monitor for important signal changes
  initial begin
    forever begin
      @(posedge crc_calculation_complete_tb);
      $display("Time %0t: CRC calculation completed, result = 0x%02h", 
               $time, crc_checksum_output_tb);
    end
  end

endmodule
```

Verilator Simulation Output:
CRC Calculator Design Module Initialized

=== CRC Calculator Testbench Starting ===

Applying reset sequence...
Starting CRC calculation with test data:
Processing byte 0: 0xab
Time 55: CRC calculation completed, result = 0x58
CRC calculation complete! Checksum: 0x58
Processing byte 1: 0xcd
Time 75: CRC calculation completed, result = 0xe2
CRC calculation complete! Checksum: 0xe2
Processing byte 2: 0xef
Time 105: CRC calculation completed, result = 0x23
CRC calculation complete! Checksum: 0x23
Processing byte 3: 0x12
Time 125: CRC calculation completed, result = 0x97
CRC calculation complete! Checksum: 0x97

Final CRC checksum result: 0x97

Testing reset functionality...
CRC after reset: 0x00

=== CRC Calculator Testbench Complete ===

Process finished with return code: 0
Removing Chapter_7_examples/example_4__crc_calculator/obj_dir directory...
Chapter_7_examples/example_4__crc_calculator/obj_dir removed successfully.


0

#### Lookup Table Function
Function implementing ROM or lookup table functionality

#### Data Conversion Functions
Functions for converting between different data types and formats

## Functions with Different Return Types

#### Complex Number Operations
Functions returning structured data for complex arithmetic operations

#### Statistical Analysis Functions
Functions returning multiple statistical measures (mean, variance, etc.)

#### Vector Operations
Functions for vector mathematics returning arrays or packed structures

#### Packet Parser Function
Function parsing network packets and returning structured information

#### Color Space Converter
Function converting between RGB, HSV, and other color representations

## Task Declarations and Calls

#### Bus Transaction Tasks
Tasks for performing read/write operations on system buses

#### Protocol Handler Tasks
Tasks implementing communication protocol sequences

#### Test Pattern Generator Task
Task generating various test patterns for verification

#### Memory Test Tasks
Tasks for comprehensive memory testing with different patterns

#### Waveform Generator Task
Task producing different types of waveforms for testing

#### File I/O Tasks
Tasks for reading from and writing to files during simulation

## Automatic vs. Static Lifetime

#### Recursive Algorithm Examples
Automatic functions implementing recursive algorithms like tree traversal

#### Call Counter Examples
Static functions maintaining call statistics and usage counters

#### Stack Implementation
Automatic functions for stack operations with proper variable scoping

#### Random Number Generator
Static function maintaining seed state between calls

#### Nested Function Calls
Examples showing variable lifetime in nested function scenarios

## Pass by Reference

#### Array Sorting Tasks
Tasks that sort arrays in-place using pass by reference

#### Matrix Operations
Functions performing matrix operations on large data structures

#### Configuration Update Tasks
Tasks modifying system configuration structures by reference

#### Buffer Management Functions
Functions managing circular buffers and memory pools

#### State Machine Tasks
Tasks updating complex state machine structures

## Return Statements in Functions

#### Input Validation Functions
Functions with multiple return points for different validation cases

#### Search Functions
Functions with early returns for efficient searching algorithms

#### Error Handling Functions
Functions demonstrating proper error detection and return patterns

#### Conditional Processing Functions
Functions with complex conditional logic and multiple exit points

## Void Functions

#### Debug and Logging Functions
Void functions for system debugging and message logging

#### Hardware Initialization Functions
Void functions for initializing hardware components and registers

#### Data Structure Maintenance
Void functions for maintaining internal data structure consistency

#### Performance Monitoring Functions
Void functions for collecting and updating performance metrics

#### Assertion Functions
Void functions implementing custom assertion and checking logic